In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from typing import  List, Union, Type, Callable

%matplotlib inline

In [2]:
df = pd.read_csv('data.csv')

In [3]:
print(len(df))
df.head()

100


,V1,V2,V3,V4,V5,V6,V7,V8,V9,Class
0,-0.33,0.69,0,1,1,0,0.8,0,0.88,1
1,-0.33,0.94,1,0,1,0,0.8,1,0.31,2
2,-0.33,0.50,1,0,0,0,1.0,-1,0.50,1
3,-0.33,0.75,0,1,1,0,1.0,-1,0.38,1
4,-0.33,0.67,1,1,0,0,0.8,-1,0.50,2


In [4]:
#This way the rare class has value 1 (for the F-score)
#df.Class = df.Class - 1

In [5]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score

from smac.configspace import ConfigurationSpace
from ConfigSpace.hyperparameters import UniformFloatHyperparameter,\
                                        UniformIntegerHyperparameter

from smac.scenario.scenario import Scenario
from smac.facade.smac_hpo_facade import SMAC4HPO #Random Forest
from smac.facade.smac_bo_facade import SMAC4BO   #Bayesian
#from smac.initial_design.random_configuration_design import RandomConfigurations
from smac.optimizer import acquisition

In [6]:
#SMAC needs a Configuration Space where it will search for the hyperparameters
cs = ConfigurationSpace()

#define the domain of each hyperparameter
lr = UniformFloatHyperparameter("learning_rate_init", 0.01, 0.1)
momentum = UniformFloatHyperparameter("momentum", 0.1, 0.9)

#add the hyperparameters to the configuration space
cs.add_hyperparameters([lr, momentum])

[learning_rate_init, Type: UniformFloat, Range: [0.01, 0.1], Default: 0.055,
 momentum, Type: UniformFloat, Range: [0.1, 0.9], Default: 0.5]

In [33]:
def MLP_score_1(conf: Union[Type[ConfigurationSpace], dict]) -> float:
    """
    PARAMETERS
    ----------
    @conf   is a ConfigurationSpace Object, but it acts as a dictionary
            so we can call a value given the key: conf['key']
            
    Returns
    -------
    @return a float value which needs to be minimized in this case cv error
    
    This function takes in the configuration and computes the error score
    using CV with 10 folds
    """
    
    MLPclf = MLPClassifier(hidden_layer_sizes=(4,2,), momentum = conf['momentum'],
                          learning_rate_init = conf['learning_rate_init'],
                          random_state=int(conf['momentum']*conf['learning_rate_init']*1000))
    
    skf = StratifiedKFold(n_splits=10, random_state=12)
    X = df.drop('Class', axis=1).values
    y = df.Class
    score = cross_val_score(MLPclf, X, y, cv=skf)
    #print(conf['learning_rate_init'], conf['momentum'], np.mean(score))
    #scores = cross_validate(MLPclf, X, y, cv=10, scoring=['accuracy','f1', 'recall', 'precision'])
    #print(conf['learning_rate_init'], conf['momentum'], np.mean(scores['test_f1']), 
    #      np.mean(scores['test_recall']), np.mean(scores['test_precision']), np.mean(scores['test_accuracy']))
    return 1 - np.mean(score)  # it needs to Minimize it!

In [34]:
scenario = Scenario(scenario={"run_obj": "quality",   # we optimize quality
                              "runcount-limit": 25,   # number iterations
                              "cs": cs,               # configuration space
                              "output_dir": "logs",    # create a log folder named 'logs'
                              "deterministic": "true"
                             },
                    cmd_options={})

INFO:smac.utils.io.cmd_reader.CMDReader:Output to logs


In [35]:
init = cs.sample_configuration(5)

In [36]:
smac_LCB = SMAC4BO(scenario=scenario, tae_runner=MLP_score_1, 
                   rng=np.random.RandomState(123),
                   initial_design=None, 
                   initial_design_kwargs={'max_config_fracs':5/25},
                   initial_configurations=init, 
                   acquisition_function=acquisition.LCB)

INFO:smac.facade.smac_bo_facade.SMAC4BO:Optimizing a deterministic scenario for quality without a tuner timeout - will make SMAC deterministic and only evaluate one configuration per iteration!
INFO:smac.initial_design.initial_design.InitialDesign:Running initial design for 5 configurations
INFO:smac.facade.smac_bo_facade.SMAC4BO:<class 'smac.facade.smac_bo_facade.SMAC4BO'>


In [37]:
#print('lr', 'momentum', 'f1', 'recall', 'precision')
result_LCB = smac_LCB.optimize()

INFO:smac.intensification.intensification.Intensifier:Challenger (0.1186) is better than incumbent (0.2119) on 1 runs.
INFO:smac.intensification.intensification.Intensifier:Changes in incumbent:
INFO:smac.intensification.intensification.Intensifier:  learning_rate_init : 0.020809293045617412 -> 0.027963957789107284
INFO:smac.intensification.intensification.Intensifier:  momentum : 0.29594653554057904 -> 0.8400125816843749
INFO:smac.stats.stats.Stats:##########################################################
INFO:smac.stats.stats.Stats:Statistics:
INFO:smac.stats.stats.Stats:#Incumbent changed: 1
INFO:smac.stats.stats.Stats:#Target algorithm runs: 4 / 25.0
INFO:smac.stats.stats.Stats:#Configurations: 4
INFO:smac.stats.stats.Stats:Used wallclock time: 3.83 / inf sec 
INFO:smac.stats.stats.Stats:Used target algorithm runtime: 3.12 / inf sec
INFO:smac.stats.stats.Stats:##########################################################
INFO:smac.facade.smac_bo_facade.SMAC4BO:Final Incumbent: None


KeyboardInterrupt: 

In [ ]:
plt.plot(smac_LCB.get_X_y()[1])

In [ ]:
smac_EI = SMAC4BO(scenario=scenario, tae_runner=MLP_score_1, 
                  rng=np.random.RandomState(123),
                  #initial_design=RandomConfigurations, 
                  #initial_design_kwargs={'rng':np.random.RandomState(2) ,'max_config_fracs':5/25},
                  initial_design=None,
                  initial_design_kwargs={'max_config_fracs':5/25},
                  initial_configurations=init, 
                  acquisition_function=acquisition.EI)

In [ ]:
print('lr', 'momentum', 'f1', 'recall', 'precision')
result_EI = smac_EI.optimize()

In [ ]:
plt.plot(smac_EI.get_X_y()[1])

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(1-np.minimum.accumulate(smac_LCB.get_X_y()[1]))
plt.plot(1-np.minimum.accumulate(smac_EI.get_X_y()[1]))
plt.legend(['LCB','EI'])
plt.ylabel('Accuracy')
plt.xlabel('Iterations')
plt.show()